In [142]:
import json, csv
import numpy as np
import pandas as pd

df = pd.read_csv('../datasets/entities_nlp_train_hongyu.csv')
df.tail(3)

,id,keyword,location,text,entities,labels,ent_dep,ent_head,ent_pos,ent_children,target
7610,10871,missing,unknown,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,"M1.94,01:04,UTC]?5,Volcano Hawaii","PERSON,TIME,CARDINAL","ROOT,nummod,compound","M1.94,km","PROPN,NOUN",[,1
7611,10872,missing,unknown,Police investigating after an e-bike collided ...,Portugal,GPE,pobj,in,ADP,Little,1
7612,10873,missing,unknown,The Latest: More Homes Razed by Northern Calif...,Northern California,LOC,None,None,None,None,1


```
Percentage of dataset with values for the specified columns:
Value for keyword: 100.0%
Value for location: 66.92%
Value for hashtag: 22.44%
Value for hashtags_have_l1_synonyms: 0.12%
Value for hashtags_have_l2_synonyms: 0.03%
Value for subj: 52.9%
Value for verb: 79.23%
Value for obj: 79.39%
Value for contains_l1_synonyms: 12.58%
Value for contains_l2_synonyms: 5.1%
Value for contains_damaged_words: 8.0%
Value for mentioned_news_org: 0.95%
Value for mentioned_relief_org: 0.04%
Value for mentions: 25.4%
Value for orgs: 59.72%
Value for gpes: 15.19%
Value for facs: 1.73%
```

In [138]:
df_alex = pd.read_parquet('../datasets/features.parquet')
df_alex.tail(3)

,id,keyword,location,text,hashtag,hashtags_have_l1_synonyms,hashtags_have_l2_synonyms,subj,verb,obj,contains_l1_synonyms,contains_l2_synonyms,contains_damaged_words,mentioned_news_org,mentioned_relief_org,mentions,orgs,gpes,facs,target
7606,10866,derailment,,Suicide bomber kills 15 in Saudi security site...,[],False,False,[bomber],[kill],"[reuters, 15, wall]",False,False,False,False,False,[],"[bomber, Reuters, World - Google News - Wall]",[],[],1
7608,10869,derailment,,Two giant cranes holding a bridge collapse int...,[],False,False,[],[hold],"[home, collapse]",True,False,True,False,False,[],[],[],[],1
7612,10873,derailment,,The Latest: More Homes Razed by Northern Calif...,[],False,False,[],[raze],[news],False,False,False,False,False,[],[],[],[],1


In [139]:
(df_alex.at[0, 'obj'])

array([], dtype=object)

In [143]:
df_alex.shape

(7503, 20)

In [144]:
df.shape

(7613, 11)

In [286]:
df_more = df_alex.drop(['id','keyword', 'location','text'], axis=1)
df_more.tail(3)

,hashtag,hashtags_have_l1_synonyms,hashtags_have_l2_synonyms,subj,verb,obj,contains_l1_synonyms,contains_l2_synonyms,contains_damaged_words,mentioned_news_org,mentioned_relief_org,mentions,orgs,gpes,facs,target
7606,[],False,False,[bomber],[kill],"[reuters, 15, wall]",False,False,False,False,False,[],"[bomber, Reuters, World - Google News - Wall]",[],[],1
7608,[],False,False,[],[hold],"[home, collapse]",True,False,True,False,False,[],[],[],[],1
7612,[],False,False,[],[raze],[news],False,False,False,False,False,[],[],[],[],1


In [267]:
df.drop(['target'], axis=1).join(df_more).dropna().head(3)

,id,keyword,location,text,entities,labels,ent_dep,ent_head,ent_pos,ent_children,...,contains_l1_synonyms,contains_l2_synonyms,contains_damaged_words,mentioned_news_org,mentioned_relief_org,mentions,orgs,gpes,facs,target
0,1,missing,unknown,Our Deeds are the Reason of this #earthquake M...,Deeds,PERSON,nsubj,are,AUX,Our,...,False,False,False,False,False,[],[],[],[],1.0
1,4,missing,unknown,Forest fire near La Ronge Sask. Canada,"La Ronge Sask,Canada","FAC,GPE",ROOT,Canada,PROPN,None,...,False,False,False,False,False,[],[],[Canada],[La Ronge Sask],1.0
2,5,missing,unknown,All residents asked to 'shelter in place' are ...,None,None,None,None,None,None,...,False,False,False,False,False,[],"[shelter, officers, other, shelter, orders]",[],[],1.0


## false

In [284]:
df.drop(['target'], axis=1).join(df_more).dropna().astype({
    'hashtags_have_l1_synonyms': 'str',
    'hashtags_have_l2_synonyms': 'str',
    'contains_l1_synonyms': 'str',
    'contains_l2_synonyms': 'str',
    'contains_damaged_words': 'str', 
    'mentioned_news_org':'str', 
    'mentioned_relief_org':'str'}).to_csv('../datasets/all-nlp-features.csv', encoding='utf-8', index=False)

## evaluation

In [219]:
df_all = df.drop(['target'], axis=1).join(df_more).dropna()

In [220]:
y = df_all['target']
y.shape

(7503,)

In [228]:
X = df_all.drop(['id', 'target'], axis=1)
X = X.astype({
    'hashtags_have_l1_synonyms': 'str',
    'hashtags_have_l2_synonyms': 'str',
    'contains_l1_synonyms': 'str',
    'contains_l2_synonyms': 'str',
    'contains_damaged_words': 'str', 
    'mentioned_news_org':'str', 
    'mentioned_relief_org':'str'})
# X = X.fillna(0)
# X = pd.DataFrame(X).fillna(0)
# # list(X.columns)
# X.head()
# df_copy = X.copy()
# df_copy.head()
# for column in df_copy:
#     df_copy[column] = df_copy[column].apply(lambda y: '' if not y else y)
#     print(df[column])
#     print(column)

# X.head()
# np.any(np.isnan(X))
X.shape

(7503, 24)

In [283]:
enc = OneHotEncoder(handle_unknown='ignore')
lb = preprocessing.LabelBinarizer()

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

mlb.fit(X['mentions'])
mlb.classes_

# lb.fit_transform(list(X['hashtags_have_l1_synonyms']))
# for data in X['hashtag']:
#     if type(data) == type(True):
#         print(data)


array(['@1061TheTwister', '@1061thetwister', '@115Film', ..., '@zeno001',
       '@zhenghxn', '@zourryart'], dtype=object)

In [289]:
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing

class LabeledNormalizer(Normalizer):
    def fit(self, X, *args, **kwargs):
        try:
            self.names = X.columns
        except:
            self.names = [str(i) for i in range(X.shape[1])]
        return super().fit(X, *args, **kwargs)
         
    def get_feature_names(self):
        return self.names
    
class SupervisionFriendlyLabelBinarizer(LabelBinarizer):
    def fit_transform(self, X, y=None):
        return super(SupervisionFriendlyLabelBinarizer,self).fit_transform(X)

class MultiLabelBinarizerWrapper(MultiLabelBinarizer):
    def fit_transform(self, X, y=None):
        return super(MultiLabelBinarizerWrapper,self).fit_transform(X)
    def get_params(self, deep=True):
        return super(MultiLabelBinarizer,self).get_params(deep=True)

# enc = OneHotEncoder(handle_unknown='ignore')
# lb = preprocessing.LabelBinarizer()

vec = ColumnTransformer([
#     ('norm', LabeledNormalizer(), ['id']),
    ('kw', TfidfVectorizer(ngram_range=(1, 1), min_df=2, token_pattern=r"(?u)\b\w+\b",), 'keyword'),
    ('loc', TfidfVectorizer(ngram_range=(1, 1), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'location'),
    ('text', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'text'),
    ('ent', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'entities'),
    ('label', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'labels'),
    ('dep', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_dep'),
    ('head', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_head'),
    ('pos', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_pos'),
    ('child', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_children'),
    ('hash', MultiLabelBinarizerWrapper(), 'hashtag'),
    ('hashl1', SupervisionFriendlyLabelBinarizer(), 'hashtags_have_l1_synonyms'),
    ('hashl2', SupervisionFriendlyLabelBinarizer(), 'hashtags_have_l2_synonyms'),
    ('subj', MultiLabelBinarizerWrapper(), 'subj'),
    ('obj', MultiLabelBinarizerWrapper(), 'verb'),
    ('verb', MultiLabelBinarizerWrapper(), 'obj'),
    ('syn1', SupervisionFriendlyLabelBinarizer(), 'contains_l1_synonyms'),
    ('syn2', SupervisionFriendlyLabelBinarizer(), 'contains_l2_synonyms'),
    ('damage', SupervisionFriendlyLabelBinarizer(), 'contains_damaged_words'),
    ('news', SupervisionFriendlyLabelBinarizer(), 'mentioned_news_org'),
    ('relief', SupervisionFriendlyLabelBinarizer(), 'mentioned_relief_org'),
    ('mentions', MultiLabelBinarizerWrapper(), 'mentions'),
#     ('orgs', MultiLabelBinarizerWrapper, 'orgs'),
#     ('gpes', MultiLabelBinarizerWrapper, 'gpes'),
#     ('facs', MultiLabelBinarizerWrapper, 'facs'),
    
])

# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(2)

# from sklearn.svm import LinearSVC
from sklearn.svm import SVC
# clf = SVC(kernel='linear', probability=True)
clf = SVC(kernel='rbf', probability=True)

# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=20, random_state=0) # use a guassian forest
# clf = RandomForestClassifier(n_estimators=200, max_features=250, random_state=0) # set limit to prevent overfitting
# clf = RandomForestClassifier(n_estimators=200, max_features=1000, random_state=0) # set limit to prevent overfitting
scaler = preprocessing.StandardScaler(with_mean=False)

# from sklearn.neural_network import MLPClassifier
# clf = MLPClassifier(solver='lbfgs', alpha=1e-5, activation='tanh', hidden_layer_sizes=(50, 100, 30), max_iter=1000, random_state=1)

pipeline = make_pipeline(vec, scaler, clf)
# pipeline = make_pipeline(vec, clf)


## evaluation

# df.contains_l1_synonyms.map(lambda x: 'true' if x else 'false')

In [290]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

def evaluate(_clf, X, y):
    report = classification_report(
        y_true=y, y_pred=cross_val_predict(pipeline, X, y, cv=5)
    )
    print(report)
    scores = cross_val_score(_clf, X, y, scoring='accuracy', cv=5)
    print('Cross-validation MSE: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    
    _clf.fit(X,y)
    print('Training Set Accuracy: {:.3f}'.format(_clf.score(X,y)))


import time
start_time = time.time()

evaluate(pipeline, X, y)

seconds = time.time() - start_time
print('\nEvaluation Time Taken: ', time.strftime("%H:%M:%S",time.gmtime(seconds)))

import time
start_time = time.time()

pipeline.fit(X,y)

seconds = time.time() - start_time
print('Training Time Taken:  ', time.strftime("%H:%M:%S",time.gmtime(seconds)))

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['#1008planet\x89Û', '#1008pla\x89Û', '#124', '#171', '#2015', '#2A', '#2MinuteMix', '#2fast2furious', '#5SOSFAM', '#7NewsAdl', '#9973', '#9newsmornings', '#AFRICANBAZE', '#ARMAGEDDON', '#AZ', '#Accident', '#Afghanistan', '#AfricansInSF', '#AirPlane', '#Aircraft', '#Airplane', '#Albany', '#AllTheNews', '#Apocalypse', '#AppreciativeInquiry', '#Armageddon', '#Arsonist', '#ArtisteOfTheWeekFact', '#Ashes', '#Ashville', '#Astrology', '#Auction', '#Avalanche', '#BINLADEN', '#BTS', '#Babies', '#Bahrain', '#BanTheBomb', '#BayArea', '#Beyhive', '#BeyondGPS', '#Bioterror', '#Bioterrorism', '#Bitcoin', '#BlackLivesMatter', '#Boing', '#Bridgetown', '#BruceWillis', '#ButGod', '#CAfire', '#CLE', '#CUFI', '#CarFest', '#Catfish', '#CatfishMTV', '#Christians', '#Civilian', '#Cleveland', '#ClubBanger', '#Collapse', '#ColoradoAvalanche', '#Conversations', '#CountyNews', '#Crash', '#CubsTalk', '#DNA'

              precision    recall  f1-score   support

         0.0       0.64      0.98      0.77      4305
         1.0       0.91      0.25      0.39      3198

    accuracy                           0.67      7503
   macro avg       0.77      0.61      0.58      7503
weighted avg       0.75      0.67      0.61      7503



/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['#1008planet\x89Û', '#1008pla\x89Û', '#124', '#171', '#2015', '#2A', '#2MinuteMix', '#2fast2furious', '#5SOSFAM', '#7NewsAdl', '#9973', '#9newsmornings', '#AFRICANBAZE', '#ARMAGEDDON', '#AZ', '#Accident', '#Afghanistan', '#AfricansInSF', '#AirPlane', '#Aircraft', '#Airplane', '#Albany', '#AllTheNews', '#Apocalypse', '#AppreciativeInquiry', '#Armageddon', '#Arsonist', '#ArtisteOfTheWeekFact', '#Ashes', '#Ashville', '#Astrology', '#Auction', '#Avalanche', '#BINLADEN', '#BTS', '#Babies', '#Bahrain', '#BanTheBomb', '#BayArea', '#Beyhive', '#BeyondGPS', '#Bioterror', '#Bioterrorism', '#Bitcoin', '#BlackLivesMatter', '#Boing', '#Bridgetown', '#BruceWillis', '#ButGod', '#CAfire', '#CLE', '#CUFI', '#CarFest', '#Catfish', '#CatfishMTV', '#Christians', '#Civilian', '#Cleveland', '#ClubBanger', '#Collapse', '#ColoradoAvalanche', '#Conversations', '#CountyNews', '#Crash', '#CubsTalk', '#DNA'

Cross-validation MSE: 0.668 ± 0.042
Training Set Accuracy: 0.965

Evaluation Time Taken:  00:14:25
Training Time Taken:   00:02:01


## rbf kernel

### add 12 more NLP features
```
              precision    recall  f1-score   support

         0.0       0.64      0.98      0.77      4305
         1.0       0.89      0.26      0.41      3198

    accuracy                           0.67      7503
   macro avg       0.76      0.62      0.59      7503
weighted avg       0.75      0.67      0.62      7503

Cross-validation MSE: 0.671 ± 0.043
Training Set Accuracy: 0.964

Evaluation Time Taken:  00:11:42
Training Time Taken:   00:01:32
```
---

### entity features
```
              precision    recall  f1-score   support

           0       0.65      0.85      0.74      4342
           1       0.67      0.40      0.50      3271

    accuracy                           0.66      7613
   macro avg       0.66      0.63      0.62      7613
weighted avg       0.66      0.66      0.64      7613

Cross-validation MSE: 0.657 ± 0.070
Training Set Accuracy: 0.902

Evaluation Time Taken: 00:09:49
Training Time Taken:   00:01:21
```
### 770M features
```
              precision    recall  f1-score   support

           0       0.65      0.87      0.74      4342
           1       0.69      0.38      0.49      3271

    accuracy                           0.66      7613
   macro avg       0.67      0.62      0.62      7613
weighted avg       0.67      0.66      0.63      7613
```

In [291]:
from sklearn.externals import joblib

pipeline.fit(X, y)

rf_21__mdl = '../models/21-nlp-features-linear-rbf.pkl'
joblib.dump(pipeline, rf_21__mdl)
!ls -lSh $rf_21__mdl

/usr/local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


-rw-r--r-- 1 hongyu hongyu 11M Mar 28 21:24 ../models/21-nlp-features-linear-rbf.pkl
